<a href="https://colab.research.google.com/github/mjavadzadeh11/Crawler/blob/master/GoogleNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GoogleNews
!pip install newspaper3k

In [3]:
import pandas as pd
from GoogleNews import GoogleNews
from newspaper import Article


In [4]:
googlenews = GoogleNews(start='11/04/2021',end='12/04/2021')
googlenews.search('Omicron')

#googlenews = GoogleNews(period='7d')
googlenews.get_page(30)
Result=googlenews.results()

In [5]:
df=pd.DataFrame(Result)
df.drop(columns=['img'], inplace=True)

In [ ]:
filt = df['media']== 'The Guardian'
df[filt]

,title,media,date,datetime,desc,link
6,Florida’s governor celebrated his anti-mandate...,The Guardian,3 hours ago,2021-12-04 10:06:49.561832,Ron DeSantis struck 'a blow for freedom' with ...,https://www.theguardian.com/us-news/2021/dec/0...
7,Sage scientists seek balanced UK response to O...,The Guardian,3 hours ago,2021-12-04 10:06:49.563737,"With Covid variant danger unclear, advisers sa...",https://www.theguardian.com/politics/2021/dec/...


In [ ]:
#filtering by media
filt = df['media']== 'Al Jazeera'
df[filt]

In [ ]:
df.iloc[0,0]

In [ ]:
#grouping by media
media_gp= df.groupby(['media'])
media_gp ['link'].value_counts() 

In [6]:
#Crawling through links:
from bs4 import BeautifulSoup
import requests

In [7]:
filt= df['media'] == 'The Guardian'

for url in df.loc[filt,'link'].values:
  source = requests.get(url).text
  cf = BeautifulSoup(source, 'lxml')
  text=[]
  for context in cf.find_all('p', class_='dcr-o5gy41'):
    text.append(context.text)
  text = ' '.join(text)

  for i in range(df.shape[0]):
    if df.loc[i,'media']=='The Guardian':
      df.loc[i,'text'] = text
    else:
      df.loc[i,'text'] = None



In [ ]:
df

In [ ]:
!pip install vaderSentiment

In [10]:
# sentiment analysis     # !pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [11]:
Analyzer = SentimentIntensityAnalyzer()
negative=[]
netrual=[]
positive=[]
for i in range(df.shape[0]):
  title = df.iloc[i,0]
  desc	 = df.iloc[i,4]
  title_analyzer = Analyzer.polarity_scores(title)
  caption_analyzer = Analyzer.polarity_scores(desc)
  negative.append(((title_analyzer['neg']) + (caption_analyzer['neg']))/2)
  positive.append(((title_analyzer['pos']) + (caption_analyzer['pos']))/2)
  netrual.append(((title_analyzer['neu']) + (caption_analyzer['neu']))/2)

df['Negative']= negative
df['Positive']= positive
df['Netrual'] = netrual

In [12]:
df.to_csv('news.csv')


In [ ]:
df.nlargest(5,['Negative'])